In [30]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import spacy

In [31]:
df_t= pd.read_csv(r'data\twitter_training1.csv')
df_t.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [32]:
columns= ['index','location','sentiment','text']
df_t.columns=columns
df= df_t
df.head()

,index,location,sentiment,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [33]:
target = {'Positive': 0,'Negative':2,'Neutral':1,'Irrelevant':3}
df['sentiment']=df['sentiment'].map(target)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      74681 non-null  int64 
 1   location   74681 non-null  object
 2   sentiment  74681 non-null  int64 
 3   text       73995 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [34]:
df = df[(df['sentiment'] != 3)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61691 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      61691 non-null  int64 
 1   location   61691 non-null  object
 2   sentiment  61691 non-null  int64 
 3   text       61120 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.4+ MB


In [35]:
df = df.dropna(subset=['text'],how='all')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61120 entries, 0 to 74680
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   index      61120 non-null  int64 
 1   location   61120 non-null  object
 2   sentiment  61120 non-null  int64 
 3   text       61120 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB


In [36]:
nlp = spacy.load('en_core_web_sm')

def preprocess(text):
    doc= nlp(text)
    process = []
    for i in doc:
        if i.is_stop or i.is_punct or i.is_space:
            continue
        process.append(i.text)
    return ' '.join(process)
    

In [37]:
# df['process_text']= df['text'].apply(preprocess)
# df.head()

In [38]:
df = pd.read_csv('processed_data.csv')

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [40]:
df.head()

,index,location,sentiment,text,process_text
0,2401,Borderlands,0,I am coming to the borders and I will kill you...,coming borders kill
1,2401,Borderlands,0,im getting on borderlands and i will kill you ...,m getting borderlands kill
2,2401,Borderlands,0,im coming on borderlands and i will murder you...,m coming borderlands murder
3,2401,Borderlands,0,im getting on borderlands 2 and i will murder ...,m getting borderlands 2 murder
4,2401,Borderlands,0,im getting into borderlands and i can murder y...,m getting borderlands murder


In [41]:
df= df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59662 entries, 0 to 61119
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         59662 non-null  int64 
 1   location      59662 non-null  object
 2   sentiment     59662 non-null  int64 
 3   text          59662 non-null  object
 4   process_text  59662 non-null  object
dtypes: int64(2), object(3)
memory usage: 2.7+ MB


In [42]:
X = df.process_text
y= df.sentiment
y.value_counts()

sentiment
2    21895
0    20146
1    17621
Name: count, dtype: int64

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)


In [44]:
clf = Pipeline([
    ('tfdf', TfidfVectorizer()),
    ('random', RandomForestClassifier())
])
clf.fit(X_train,y_train)
y_pred_ran = clf.predict(X_test)
print(classification_report(y_test,y_pred_ran))


              precision    recall  f1-score   support

           0       0.92      0.95      0.93      4030
           1       0.95      0.92      0.94      3524
           2       0.94      0.94      0.94      4379

    accuracy                           0.94     11933
   macro avg       0.94      0.94      0.94     11933
weighted avg       0.94      0.94      0.94     11933



In [45]:
y_test[:5]

11797    2
48232    2
18255    2
33868    0
19960    1
Name: sentiment, dtype: int64

In [46]:
y_pred_ran[:5]

array([2, 2, 2, 0, 1], dtype=int64)

In [48]:
from sklearn.naive_bayes import MultinomialNB
clf_naive = Pipeline([
    ('tfdf', TfidfVectorizer()),
    ('naive', MultinomialNB())
])
clf_naive.fit(X_train,y_train)
y_pred_naive = clf_naive.predict(X_test)
print(classification_report(y_test,y_pred_naive))


              precision    recall  f1-score   support

           0       0.81      0.85      0.83      4030
           1       0.89      0.68      0.77      3524
           2       0.78      0.90      0.83      4379

    accuracy                           0.82     11933
   macro avg       0.83      0.81      0.81     11933
weighted avg       0.82      0.82      0.81     11933



In [47]:
import joblib
joblib.dump(clf, 'text_classification_model.joblib')

print("Model saved successfully.")

Model saved successfully.
